In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

In [2]:
df = pd.read_csv('total_data.csv')

### xdata로 상관계수가 높은 column을 넣어서 Elastic
- elasticnet으로 상관계수가 높은 feature를 넣어 모델생성

In [3]:
# 필요 패키지 로드
#from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error

In [4]:
# y값인 q1-q5가 결측인 2020년 데이터 제거
a = df[0:-82]
a

,jur_stn,year,child,house,report_sx_total,vio_cnt,mur_rob_cnt,car_cnt,popu_합계,for_합계,...,vic_w,vic_unknown,case_car_man,case_car_car,case_car_only,q1,q2,q3,q4,q5
0,서울중부경찰서,2017(상반기),39.0,97.0,22574.0,1324.0,0.0,1412.0,63948.0,5132.5,...,0.241636,0.022305,0.274074,0.703704,0.022222,75.2,76.5,69.9,65.9,72.6
1,서울종로경찰서,2017(상반기),46.0,98.0,19674.0,1102.0,0.0,867.0,78638.5,5489.5,...,0.262745,0.039216,0.282353,0.682353,0.035294,81.6,82.4,69.1,62.7,72.2
2,서울남대문경찰서,2017(상반기),39.0,97.0,14499.0,949.0,0.0,736.0,63948.0,5132.5,...,0.203922,0.043137,0.247059,0.713725,0.039216,77.3,79.3,68.8,61.0,71.7
3,서울서대문경찰서,2017(상반기),39.0,98.0,38769.0,1924.0,0.0,1788.0,321345.0,13152.0,...,0.243697,0.044118,0.266807,0.691176,0.042017,78.0,80.0,67.0,66.3,71.4
4,서울혜화경찰서,2017(상반기),46.0,98.0,18857.0,1127.0,0.0,899.0,78638.5,5489.5,...,0.217054,0.046512,0.262548,0.694981,0.042471,78.6,80.6,70.7,65.2,75.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,창원서부경찰서,2019(하반기),44.0,87.0,22771.0,1460.0,2.0,1964.0,256001.0,5100.0,...,0.286982,0.047337,0.218935,0.733728,0.047337,79.1,79.5,68.5,71.2,76.0
242,마산중부경찰서,2019(하반기),37.0,88.0,21264.0,1265.0,5.0,1584.0,177579.0,3145.0,...,0.280000,0.092308,0.258462,0.649231,0.092308,80.8,80.0,70.8,73.2,76.5
243,마산동부경찰서,2019(하반기),44.0,86.0,26014.0,1715.0,8.0,1910.0,192446.0,2871.0,...,0.345679,0.032922,0.345679,0.621399,0.032922,75.0,76.5,69.4,68.9,73.1
244,진주경찰서,2019(하반기),64.0,92.0,38557.0,2552.0,0.0,3570.0,354166.0,7152.0,...,0.357320,0.057072,0.357320,0.588089,0.054591,78.7,79.6,70.3,74.6,75.6


In [5]:
# 경찰서와 연도 데이터 제거
a.drop(columns = ['jur_stn', 'year'], inplace = True)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [6]:
a_ = a.drop(columns = ['q1', 'q2', 'q3', 'q4', 'q5'])

In [7]:
a_

,child,house,report_sx_total,vio_cnt,mur_rob_cnt,car_cnt,popu_합계,for_합계,popu_u20,popu_o60,...,vic_20,vic_340,vic_5670,vic_80ov,vic_m,vic_w,vic_unknown,case_car_man,case_car_car,case_car_only
0,39.0,97.0,22574.0,1324.0,0.0,1412.0,63948.0,5132.5,8387.0,14108.0,...,0.129278,0.349810,0.471483,0.019011,0.736059,0.241636,0.022305,0.274074,0.703704,0.022222
1,46.0,98.0,19674.0,1102.0,0.0,867.0,78638.5,5489.5,11965.5,16858.0,...,0.129555,0.404858,0.429150,0.024291,0.698039,0.262745,0.039216,0.282353,0.682353,0.035294
2,39.0,97.0,14499.0,949.0,0.0,736.0,63948.0,5132.5,8387.0,14108.0,...,0.111111,0.399177,0.465021,0.012346,0.752941,0.203922,0.043137,0.247059,0.713725,0.039216
3,39.0,98.0,38769.0,1924.0,0.0,1788.0,321345.0,13152.0,49895.0,65887.0,...,0.167033,0.342857,0.430769,0.021978,0.712185,0.243697,0.044118,0.266807,0.691176,0.042017
4,46.0,98.0,18857.0,1127.0,0.0,899.0,78638.5,5489.5,11965.5,16858.0,...,0.141700,0.340081,0.485830,0.020243,0.736434,0.217054,0.046512,0.262548,0.694981,0.042471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,44.0,87.0,22771.0,1460.0,2.0,1964.0,256001.0,5100.0,44789.0,49023.0,...,0.111801,0.378882,0.456522,0.024845,0.665680,0.286982,0.047337,0.218935,0.733728,0.047337
242,37.0,88.0,21264.0,1265.0,5.0,1584.0,177579.0,3145.0,28095.0,50991.0,...,0.091525,0.355932,0.498305,0.033898,0.627692,0.280000,0.092308,0.258462,0.649231,0.092308
243,44.0,86.0,26014.0,1715.0,8.0,1910.0,192446.0,2871.0,32856.0,47491.0,...,0.140426,0.340426,0.434043,0.029787,0.621399,0.345679,0.032922,0.345679,0.621399,0.032922
244,64.0,92.0,38557.0,2552.0,0.0,3570.0,354166.0,7152.0,66710.0,79729.0,...,0.134211,0.310526,0.465789,0.036842,0.585608,0.357320,0.057072,0.357320,0.588089,0.054591


In [8]:
a_1 = a_

In [9]:
a_2 = a_

In [10]:
a_3 = a_

In [11]:
a_4 = a_

In [12]:
a_5 = a_

In [13]:
# StandardScaling
scaler1 = StandardScaler()
scaler1.fit(a_1) # scaler에 xdata 학습
a_s1 = scaler1.transform(a_1)
a_s1

array([[-1.16204451, -1.00794068, -0.94618196, ...,  0.39434438,
        -0.11566933, -0.96802532],
       [-0.81221223, -0.8173539 , -1.12728457, ...,  0.56656445,
        -0.52850194,  0.00726295],
       [-1.16204451, -1.00794068, -1.45045906, ..., -0.16763689,
         0.07810924,  0.29984943],
       ...,
       [-0.91216431, -3.10439533, -0.73135679, ...,  1.88389689,
        -1.70708755, -0.16973381],
       [ 0.08735649, -1.96087461,  0.05194325, ...,  2.1260591 ,
        -2.35115781,  1.44696268],
       [-1.26199659, -2.53263497, -1.0869424 , ..., -1.10883527,
         0.83430992,  0.7576378 ]])

In [14]:
# StandardScaling
scaler2 = StandardScaler()
scaler2.fit(a_2) # scaler에 xdata 학습
a_s2 = scaler2.transform(a_2)
a_s2

array([[-1.16204451, -1.00794068, -0.94618196, ...,  0.39434438,
        -0.11566933, -0.96802532],
       [-0.81221223, -0.8173539 , -1.12728457, ...,  0.56656445,
        -0.52850194,  0.00726295],
       [-1.16204451, -1.00794068, -1.45045906, ..., -0.16763689,
         0.07810924,  0.29984943],
       ...,
       [-0.91216431, -3.10439533, -0.73135679, ...,  1.88389689,
        -1.70708755, -0.16973381],
       [ 0.08735649, -1.96087461,  0.05194325, ...,  2.1260591 ,
        -2.35115781,  1.44696268],
       [-1.26199659, -2.53263497, -1.0869424 , ..., -1.10883527,
         0.83430992,  0.7576378 ]])

In [15]:
# StandardScaling
scaler3 = StandardScaler()
scaler3.fit(a_3) # scaler에 xdata 학습
a_s3 = scaler3.transform(a_3)
a_s3

array([[-1.16204451, -1.00794068, -0.94618196, ...,  0.39434438,
        -0.11566933, -0.96802532],
       [-0.81221223, -0.8173539 , -1.12728457, ...,  0.56656445,
        -0.52850194,  0.00726295],
       [-1.16204451, -1.00794068, -1.45045906, ..., -0.16763689,
         0.07810924,  0.29984943],
       ...,
       [-0.91216431, -3.10439533, -0.73135679, ...,  1.88389689,
        -1.70708755, -0.16973381],
       [ 0.08735649, -1.96087461,  0.05194325, ...,  2.1260591 ,
        -2.35115781,  1.44696268],
       [-1.26199659, -2.53263497, -1.0869424 , ..., -1.10883527,
         0.83430992,  0.7576378 ]])

In [16]:
# StandardScaling
scaler4 = StandardScaler()
scaler4.fit(a_4) # scaler에 xdata 학습
a_s4 = scaler4.transform(a_4)
a_s4

array([[-1.16204451, -1.00794068, -0.94618196, ...,  0.39434438,
        -0.11566933, -0.96802532],
       [-0.81221223, -0.8173539 , -1.12728457, ...,  0.56656445,
        -0.52850194,  0.00726295],
       [-1.16204451, -1.00794068, -1.45045906, ..., -0.16763689,
         0.07810924,  0.29984943],
       ...,
       [-0.91216431, -3.10439533, -0.73135679, ...,  1.88389689,
        -1.70708755, -0.16973381],
       [ 0.08735649, -1.96087461,  0.05194325, ...,  2.1260591 ,
        -2.35115781,  1.44696268],
       [-1.26199659, -2.53263497, -1.0869424 , ..., -1.10883527,
         0.83430992,  0.7576378 ]])

In [17]:
# StandardScaling
scaler5 = StandardScaler()
scaler5.fit(a_5) # scaler에 xdata 학습
a_s5 = scaler5.transform(a_5)
a_s5

array([[-1.16204451, -1.00794068, -0.94618196, ...,  0.39434438,
        -0.11566933, -0.96802532],
       [-0.81221223, -0.8173539 , -1.12728457, ...,  0.56656445,
        -0.52850194,  0.00726295],
       [-1.16204451, -1.00794068, -1.45045906, ..., -0.16763689,
         0.07810924,  0.29984943],
       ...,
       [-0.91216431, -3.10439533, -0.73135679, ...,  1.88389689,
        -1.70708755, -0.16973381],
       [ 0.08735649, -1.96087461,  0.05194325, ...,  2.1260591 ,
        -2.35115781,  1.44696268],
       [-1.26199659, -2.53263497, -1.0869424 , ..., -1.10883527,
         0.83430992,  0.7576378 ]])

In [18]:
# x데이터 설정 2017, 2018 데이터를 학습용, 2019년 데이터를 검증용 데이터셋으로 설정

xtrain1 = a_s2[:-82]
xtest1 = a_s2[-82:]
xtrain2 = a_s2[:-82]
xtest2 = a_s2[-82:]
xtrain3 = a_s3[:-82]
xtest3 = a_s3[-82:]
xtrain4 = a_s4[:-82]
xtest4 = a_s4[-82:]
xtrain5 = a_s5[:-82]
xtest5 = a_s5[-82:]

In [19]:
# y데이터 설정 2017, 2018 데이터를 학습용, 2019년 데이터를 검증용 데이터셋으로 설정

train = a[:-82]
test = a[-82:]

ytrain1 = train['q1']
ytrain2 = train['q2']
ytrain3 = train['q3']
ytrain4 = train['q4']
ytrain5 = train['q5']

ytest1 = test['q1']
ytest2 = test['q2']
ytest3 = test['q3']
ytest4 = test['q4']
ytest5 = test['q5']

In [20]:
a_.columns

Index(['child', 'house', 'report_sx_total', 'vio_cnt', 'mur_rob_cnt',
       'car_cnt', 'popu_합계', 'for_합계', 'popu_u20', 'popu_o60', 'for_u20',
       'for_2030', 'single_합계', 'single_u20', 'single_2030', 'single_4050',
       'single_o60', 'single_f_합계', 'single_f_u20', 'single_f_2030',
       'single_f_4050', 'single_f_o60', 'popu_2050', 'for_o40', '총범죄발생대비검거수',
       '절도폭력범죄발생대비검거수', '강도살인범죄발생대비검거수', '교통사고범죄발생대비검거수', '기초질서범죄발생대비검거수',
       '인구수대비검거수', '외국인인구수대비검거수', '인구수대비경찰수', 'before_q1', 'before_q2',
       'before_q3', 'before_q4', 'before_q5', 'light_r', 'cctv_r', 'adult_r',
       'bell_r', 'ofn_10', 'ofn_20', 'ofn_340', 'ofn_5670', 'ofn_80ov',
       'ofn_m', 'ofn_w', 'ofn_unknown', 'vic_10', 'vic_20', 'vic_340',
       'vic_5670', 'vic_80ov', 'vic_m', 'vic_w', 'vic_unknown', 'case_car_man',
       'case_car_car', 'case_car_only'],
      dtype='object')

In [21]:
# 그리드 서치 패키지
from sklearn.model_selection import GridSearchCV

In [22]:
# 최적 성능을 내는 lasso의 alpha값을 얻기 위해 param_grid 생성

param_grid = {'l1_ratio': np.linspace(0, 1.0, 101), 'alpha' : np.linspace(0.001, 1.0, 1000)}

#그리드 서치 설정
#grid_search = GridSearchCV(ElasticNet(), param_grid = param_grid, cv = 10, n_jobs = -1, scoring ='r2')
grid_search = GridSearchCV(ElasticNet(), param_grid = param_grid, cv = 10, n_jobs = -1, scoring ='neg_mean_absolute_error')

#### q1 절도폭력 

In [23]:
# 그리드 서치 후 최고 성능의 모델을 ela1에 저장

grid_search.fit(xtrain1, ytrain1)
ela1 = grid_search.best_estimator_

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 456.2833853570732, tolerance: 0.19098478048780493
  model = cd_fast.enet_coordinate_descent(


In [24]:
# MAE 출력

y_pred1 = ela1.predict(xtest1)
mean_absolute_error(ytest1, y_pred1)

2.292190233585515

In [25]:
# 결과

print('alpha =', ela1.alpha)
print('l1_ratio =', ela1.l1_ratio)
print(ela1.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 개수:', len(a_1.columns[ela1.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_1.columns[ela1.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_1.columns[ela1.coef_.argmin()])

alpha = 0.455
l1_ratio = 0.0
[ 0.29055879  0.20573819 -0.06336278 -0.30400832  0.02153581  0.02599198
  0.07542275 -0.22136329  0.20991095  0.0689101   0.07959131 -0.15830058
 -0.05603413  0.25742525  0.05963968 -0.27949482 -0.15329153  0.07989102
  0.21115002  0.16605632 -0.07230183 -0.04327042  0.03726423 -0.26389192
 -0.150684   -0.26842675 -0.04660692  0.0437711   0.21405164 -0.08429702
  0.4407271   0.19185572  0.20524433  0.15122419 -0.21552418  0.21269901
  0.21354654  0.16352937 -0.21110401 -0.19525778  0.11518121 -0.23795511
 -0.16786872  0.13270439  0.06040047 -0.05709478 -0.24859332  0.19744365
  0.15047296 -0.32443889 -0.0498592   0.23662695 -0.05593132 -0.13815488
 -0.18979382  0.21164893 -0.01035868  0.00557629 -0.01489794  0.03748586]
feature 개수: 60
가장 강한 양의 상관관계:  외국인인구수대비검거수 
가장 강한 음의 상관관계:  vic_10


#### q2 강도살인

In [26]:
# 그리드 서치 후 최고 성능의 모델을 ela2에 저장

grid_search.fit(xtrain2, ytrain2)
ela2 = grid_search.best_estimator_

In [27]:
# MAE 출력

y_pred2 = ela2.predict(xtest2)
mean_absolute_error(ytest2, y_pred2)

2.048285407496436

In [28]:
# 결과

print('alpha =', ela2.alpha)
print('l1_ratio =', ela2.l1_ratio)
print(ela2.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 개수:', len(a_2.columns[ela2.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_2.columns[ela2.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_2.columns[ela2.coef_.argmin()])

alpha = 0.021
l1_ratio = 0.96
[ 0.          0.43015483 -0.         -2.00653231  0.          0.74352238
  0.         -0.          0.59240671  1.90828355  1.7693879  -1.02044052
  0.          0.37423577  0.12561828 -1.23191412 -0.59158367  0.
 -0.          0.87551068  0.         -0.          0.         -0.53113007
 -0.         -0.         -0.          0.06635165  0.19939797 -0.9352667
  0.71466768  1.37086654 -0.14037449  0.         -0.53665128  0.28707407
  0.46273082  0.10376233 -0.15726244 -0.19116325  0.0155787  -0.61491058
 -0.          0.30392962 -0.         -0.         -0.89212954  0.
  0.16228091  0.         -0.06554546  0.          0.         -0.59566596
 -0.          0.04686557 -0.          0.         -0.45946286  0.28547933]
feature 개수: 35
가장 강한 양의 상관관계:  popu_o60 
가장 강한 음의 상관관계:  vio_cnt


In [50]:
# q2 관련 feature
print('상위 30개 feature\n', a_2.columns[sorted(range(len(ela2.coef_)), key=lambda i: abs(ela2.coef_[i]))[-30:]])

상위 30개 feature
 Index(['피해_80대이상', 'robber_murder_y', 'report_sx_여성', 'robber_murder_x',
       'theft_violence_y', '가해_10대이하', 'for_합계', 'for_2030', 'for_u20',
       'theft_violence_x', 'vio_cnt', 'single_f_4050', 'single_f_o60',
       'single_4050', 'for_4050', 'single_f_합계', 'for_o60', 'popu_o60',
       'single_합계', 'single_o60'],
      dtype='object')


#### q3 교통안전

In [ ]:
# 그리드 서치 후 최고 성능의 모델을 lasso3에 저장

grid_search.fit(xtrain3, ytrain3)
ela3 = grid_search.best_estimator_

In [ ]:
# MAE 출력

y_pred3 = ela3.predict(xtest3)
mean_absolute_error(ytest3, y_pred3)

In [ ]:
# 결과

print('alpha =', ela3.alpha)
print('l1_ratio =', ela3.l1_ratio)
print(ela3.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 개수:', len(a_3.columns[ela3.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_3.columns[ela3.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_3.columns[ela3.coef_.argmin()])

In [54]:
# q3 관련 feature
print('상위 30개 feature\n', a_3.columns[sorted(range(len(ela3.coef_)), key=lambda i: abs(ela3.coef_[i]))[-30:]])

상위 30개 feature
 Index(['robber_murder_y', 'single_f_2030', '가해_80대이상', 'single_2030', 'adult',
       'robber_murder_x', 'mur_rob_cnt', 'theft_violence_y', 'single_합계',
       'for_o60', 'theft_violence_x', '5m_crm_yn', 'for_합계', 'bell',
       'for_4050', 'for_2030', 'traffic_x', '가해_70대', '피해_70대', 'single_4050'],
      dtype='object')


#### q4 법질서 준수도

In [ ]:
# 그리드 서치 후 최고 성능의 모델을 lasso4에 저장

grid_search.fit(xtrain4, ytrain4)
ela4 = grid_search.best_estimator_

In [ ]:
# MAE 출력

y_pred4 = ela4.predict(xtest4)
mean_absolute_error(ytest4, y_pred4)

In [ ]:
# 결과

print('alpha =', ela4.alpha)
print('l1_ratio =', ela4.l1_ratio)
print(ela4.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 개수:', len(a_4.columns[ela4.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_4.columns[ela4.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_4.columns[ela4.coef_.argmin()])

In [58]:
# q3 관련 feature
print('상위 30개 feature\n', a_4.columns[sorted(range(len(ela4.coef_)), key=lambda i: abs(ela4.coef_[i]))[-30:]])

상위 30개 feature
 Index(['cctv', 'for_o60', '가해_여', 'mur_rob_cnt', 'single_합계', 'single_4050',
       'car_cnt', 'for_2030', 'theft_violence_x', 'for_4050', 'for_합계'],
      dtype='object')


#### q5 전반적 안전도

In [ ]:
# 그리드 서치 후 최고 성능의 모델을 lasso4에 저장

grid_search.fit(xtrain5, ytrain5)
ela5 = grid_search.best_estimator_

In [ ]:
# MAE 출력

y_pred5 = ela5.predict(xtest5)
mean_absolute_error(ytest5, y_pred5)

In [ ]:
# 결과

print('alpha =', ela5.alpha)
print('l1_ratio =', ela5.l1_ratio)
print(ela5.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 개수:', len(a_5.columns[ela5.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_5.columns[ela5.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_5.columns[ela5.coef_.argmin()])

In [ ]:
print('feature: ', a_5.columns[ela5.coef_ != 0])

In [ ]:
# q5 관련 feature
print('상위 30개 feature\n', a_5.columns[sorted(range(len(ela5.coef_)), key=lambda i: abs(ela5.coef_[i]))[-30:]])